# init

In [ ]:
import json
import pickle
import numpy as np
from datetime import datetime
from wsgiref.util import setup_testing_defaults
from wsgiref.simple_server import make_server

In [ ]:
PORT = 8000

In [ ]:
class ModelPredictor:
    """this class is body persing and predict.

    attributes:
        model(:obj:): model.

    """
    
    def __init__(self):
        """init predictor

        load model

        """
        self.model = pickle.load(open('model', 'rb'))


    def main(self,data):
        """main function
    
        Args:
           obj(object:dict): dict object.
    
        Returns:
            byte object

        Examples:
            >>> ModelPredictor().main({"sepal length (cm)":1,"sepal width (cm)":2,"petal length (cm)":3,"petal width (cm)":4})
            b{"val": 2}
        """
        return json.dumps({"val": self.predict(self.prep(data))} ,cls = NumpyEncoder).encode()
    

    def prep(self,data):
        """preparate requested data
    
        Args:
           data (object:dict): dict object.
    
        Returns:
            2 dimension list

        Examples:
            >>> ModelPredictor().prep({"sepal length (cm)":1,"sepal width (cm)":2,"petal length (cm)":3,"petal width (cm)":4})
            [[1,2,3,4]]
        """
        return [[   data['sepal length (cm)'],
                    data['sepal width (cm)'],
                    data['petal length (cm)'],
                    data['petal width (cm)']]]


    def predict(self,data):
        """predict from recieve data
    
        predict from recieve data
    
        Args:
           recieve(obj:): http recieved object
    
        Returns:
            bytes: byte object

        Examples:
            >>> ModelPredictor().predict([[1,2,3,4]])
            2
        """
        return self.model.predict(data)[0]

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """numpy data encode

    attributes:
        model(:obj:): model.

    """


    def default(self, obj):
        """validate object
    
        Args:
           obj(np.object): numpy object.
    
        Returns:
            (obj: int | float | list): object

        """
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)

In [ ]:
prd_controller = ModelPredictor()

# create prediction server

In [ ]:
def simple_app(environ, start_response):
    """
    
    Args:
        obj(np.object): numpy object.

    Returns:
        (obj: int | float | list): object
    
    """
    setup_testing_defaults(environ)
    
    wsgi_input = environ["wsgi.input"]
    content_length = int(environ["CONTENT_LENGTH"])
    data = json.loads(wsgi_input.read(content_length))
    print(wsgi_input)
    print(data)
    status = '200 OK'
    headers = [('Content-type', 'text/plain; charset=utf-8')]
    ret = [prd_controller.main(data)]
    start_response(status, headers)

    return ret

In [ ]:
with make_server('0.0.0.0', PORT, simple_app) as httpd:
    print("Serving on port ${PORT}...")
    httpd.serve_forever()